# BASED COINBASE BOT TRADER

FILENAME: model_trader.ipynb
    
DATE UPDATE: 6-MAR-21
    
VERSION: 0.1

## PHASE 1: Environment Setup

Import the necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import statistics
import plotly
import plotly.express as px
import getpass
import json as js
from datetime import datetime, timedelta
# IMPORTS
from coinbase.wallet.client import Client
from coinbase.wallet.model import APIObject

import gspread
from oauth2client.service_account import ServiceAccountCredentials

import requests
from requests.exceptions import HTTPError


# machine learning libraries
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from matplotlib import pyplot as plt

# Uncomment this if you like to use the old MPL library
#from mpl_finance import candlestick_ohlc

import mplfinance as mpf
import matplotlib.dates as mpl_dates
import matplotlib.ticker as tkr
%matplotlib inline
import seaborn as sns

import dash
import dash_core_components as dcc
import dash_html_components as html

import cbpro
import json, hmac, hashlib, time, requests
from requests.auth import AuthBase

#### FUNCTION DECLARATION

In [ ]:
def connect(url, *args, **kwargs):
    try:
        if kwargs.get('param', None) is not None:
            response = requests.get(url,params)
        else:
            response = requests.get(url)
        response.raise_for_status()
        #print('HTTP connection success!')
        return response
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

In [ ]:
def is_ma_pos(val):
    
    if val > 0:
        return True
    else:
        return False

In [ ]:
def numpy_ewma_vectorized(data, window):

    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha

    scale = 1/alpha_rev
    n = data.shape[0]

    r = np.arange(n)
    scale_arr = scale**r
    offset = data[0]*alpha_rev**(r+1)
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]
    return out

Before implementation, set environmental variables with the names API_KEY and API_SECRET

In [ ]:

API_KEY = getpass.getpass(prompt='Please enter the API Key: ', stream=None)
API_SECRET = getpass.getpass(prompt='Please enter the API Secret: ', stream=None)
passphrase = getpass.getpass(prompt='Please enter the API passphrase: ', stream=None)

import cbpro
auth_client = cbpro.AuthenticatedClient(API_KEY, API_SECRET, passphrase)

# Use the sandbox API (requires a different set of API access credentials)
#auth_client = cbpro.AuthenticatedClient(API_KEY, API_SECRET, passphrase, api_url="https://api-public.sandbox.pro.coinbase.com")

### Establish Coinbase API connection

Connect to the Coinbase Pro API

In [ ]:
REST_API = 'https://api.pro.coinbase.com'
PRODUCTS = REST_API+'/products'

response = connect(PRODUCTS)
response_content = response.content
response_text = response.text
response_headers = response.headers

Display the response headers

In [ ]:
response_headers

I am only interested in a few currencies that I want to trade, so let's add them here:

### Algorand Test Section

In [ ]:

MY_CURRENCIES = ['ALGO-USD']

df_currencies = pd.read_json (response_text)

print("\nNumber of columns in the dataframe: %i" % (df_currencies.shape[1]))
print("Number of rows in the dataframe: %i\n" % (df_currencies.shape[0]))

df_currencies['id']

Print list of column names

In [ ]:
list(df_currencies.columns)

In [ ]:
columns = list(df_currencies.columns)
print(columns)
print() 
df_currencies[df_currencies.id.isin(MY_CURRENCIES)][['id', 'quote_currency', 'base_min_size', 'base_max_size']].head(5)

Display the currency info

In [ ]:
currency_rows = []
for currency in MY_CURRENCIES:
    response = connect(PRODUCTS+'/'+currency+'/stats')
    response_content = response.content
    data = js.loads(response_content.decode('utf-8'))
    currency_rows.append(data)
    
# Create dataframe and set row index as currency name
df_statistics = pd.DataFrame(currency_rows, index = MY_CURRENCIES)
df_statistics

Retrieve the last 300 days worth of data

In [ ]:
start_date = (datetime.today() - timedelta(days=300)).isoformat()
end_date = datetime.now().isoformat()

# Please refer to the coinbase documentation on the expected parameters
params = {'start':start_date, 'end':end_date, 'granularity':'86400'}
response = connect(PRODUCTS+'/ALGO-USD/candles', param = params)
response

In [ ]:
response_text = response.text

Add column names in line with the Coinbase Pro documentation

In [ ]:
df_history = pd.read_json(response_text)


df_history.columns = ['time','low','high','open','close','volume']
df_history

We will add a few more columns just for better readability

In [ ]:
df_history['date'] = pd.to_datetime(df_history['time'], unit='s')
df_history['year'] = pd.DatetimeIndex(df_history['date']).year
df_history['month'] = pd.DatetimeIndex(df_history['date']).month
df_history['day'] = pd.DatetimeIndex(df_history['date']).day
df_history['hour'] = pd.DatetimeIndex(df_history['date']).hour.astype(str)
# Only display the first 5 rows
df_history.tail(5)

In [ ]:
df_history.describe()

In [ ]:
algo_close_values = df_history['close'].tolist()
algo_close_values

### Algorand Smoothing Average Calculation

Append new columns to the dataframe

In [ ]:
algo_INVESTMENT = 100 # initial investment
algo_NO_OF_RECORDS = 0 # number of transactions
algo_ATH = algo_close_values[0]
algo_ATL = algo_close_values[0]
algo_MEDIAN = 0
algo_AVERAGE = 0

In [ ]:
algo_aggregated_list = []
algo_aggregate_df = pd.DataFrame(columns=['Close','ATH','ATL','Median','Mean'])

for value in algo_close_values:
        
    algo_aggregated_list.append(value)
    
    algo_no_of_items = len(algo_aggregated_list)
    
    if value > algo_ATH:
        algo_ATH = value
    elif value < algo_ATL:
        algo_ATL = value
    
    algo_median_val = statistics.median(algo_aggregated_list)
    algo_avg_val = statistics.mean(algo_aggregated_list)
    print('At index number: {}'.format(algo_no_of_items))
    
    print("ATH: {}, ATL: {}, Running Median: {}, Running Mean: {}\n".format(algo_ATH,algo_ATL,algo_median_val,algo_avg_val))
    
    algo_aggregate_df.loc[algo_no_of_items] = [value, algo_ATH ,algo_ATL, algo_median_val, algo_avg_val]

Add smoothing moving averages of 7 and 30 days

In [ ]:
algo_aggregate_df['SMA7'] = algo_aggregate_df['Close'].rolling(window=7).mean()
algo_aggregate_df['SMA30'] = algo_aggregate_df['Close'].rolling(window=30).mean()

algo_aggregate_df

Visualize the SMA7 and SMA30 data

In [ ]:
fig = px.line(algo_aggregate_df, x=algo_aggregate_df.index, y=['Close','SMA7','SMA30'])
fig.show()

Calculate the difference from the SMA30 and SMA7 values

In [ ]:
algo_aggregate_df['diff_SMA7_SMA30'] = algo_aggregate_df['SMA7'] - algo_aggregate_df['SMA30']
algo_aggregate_df

Create a new boolean column to determine whether the SMA30-SMA7 value is positive or negative

In [ ]:
algo_aggregate_df['bool_diff_SMA7_SMA30'] = algo_aggregate_df['diff_SMA7_SMA30'].apply(is_ma_pos)
algo_aggregate_df[75:100]

#### Calculate the MACD values for each record

In [ ]:
algo_aggregate_df['S-MACD'] = algo_aggregate_df['SMA7']/algo_aggregate_df['SMA30']
fig = px.line(algo_aggregate_df, x=algo_aggregate_df.index, y=['S-MACD'])
fig.show()

Plot the subplot graph

In [ ]:


from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=algo_aggregate_df.index, y=algo_aggregate_df['Close'], name="yaxis data"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=algo_aggregate_df.index, y=algo_aggregate_df['S-MACD'], name="yaxis2 data"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Double Y Axis Example"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="<b>primary</b> yaxis title", secondary_y=False)
fig.update_yaxes(title_text="<b>secondary</b> yaxis title", secondary_y=True)

fig.show()

Plot candlestick chart

In [ ]:
# Make a copy of the original dataframe
df_ohlc = df_history
# Remove unnecessary columns and only show the last 30 days
df_ohlc = df_ohlc.drop(['time','year','month','day'], axis = 1).head(30)
# Columns must be in a specific order for the candlestick chart (OHLC)
df_ohlc = df_ohlc[['date', 'open', 'high', 'low', 'close','volume']]
# Index must be set as the date
df_ohlc.set_index('date', inplace=True)
# Inverse order is expected so let's reverse the rows in the dataframe
df_ohlc = df_ohlc[::-1]
mpf.plot(df_ohlc,type='candle',mav=(3,6,9),volume=True)

### Exponential Moving Average Test Area

In [ ]:
algo_aggregate_df.head(5)

In [ ]:
close_array = algo_aggregate_df['Close'].to_numpy()
close_array

In [ ]:
EMA10 = numpy_ewma_vectorized(close_array,10)
EMA10

## PHASE 3: Class Initialization

In [ ]:
set(df_currencies['id'])

In [ ]:
class token:
    
    def __init__(self, name, num_of_days):
        
        print("Creating {} object ...".format(name))
        self.REST_API = 'https://api.pro.coinbase.com'
        self.PRODUCTS = REST_API+'/products'
        
        self.token_name = name
        self.token_url = '/'+name+'/candles'
        
        self.start_date = (datetime.today() - timedelta(days=300)).isoformat()
        self.end_date = datetime.now().isoformat()
        
        # Please refer to the coinbase documentation on the expected parameters
        self.params = {'start':self.start_date, 'end':self.end_date, 'granularity':'86400'}
        self.response = connect(PRODUCTS+ self.token_url, param = params)
        
        self.response_text = self.response.text
        
        self.raw_df = pd.read_json(self.response_text)
        
        self.raw_df.columns = ['time','low','high','open','close','volume']
        self.raw_df['date'] = pd.to_datetime(self.raw_df['time'], unit='s')
        self.raw_df['year'] = pd.DatetimeIndex(self.raw_df['date']).year
        self.raw_df['month'] = pd.DatetimeIndex(self.raw_df['date']).month
        self.raw_df['day'] = pd.DatetimeIndex(self.raw_df['date']).day
        self.raw_df['hour'] = pd.DatetimeIndex(self.raw_df['date']).hour.astype(str)
        self.raw_df['name'] = self.token_name
        #self.raw_df.set_index('date', inplace=True)
        
        self.sma_signal = 7
        self.sma_ma = 25
        self.ema_signal = 12
        self.ema_ma = 26
        self.sma_model_accuracy = 0
        self.ema_model_accuracy = 0
        
    def return_df(self):
        
        return self.aggregate_df
    

    def calc_sma(self):
        
        close_values = self.raw_df['close'][::-1].tolist()
    
        
        INVESTMENT = 100 # initial investment
        NO_OF_RECORDS = 0 # number of transactions
        ATH = algo_close_values[0]
        ATL = algo_close_values[0]
        MEDIAN = 0
        AVERAGE = 0
        
        aggregated_list = []
        self.aggregate_df = pd.DataFrame(columns=['Close','ATH','ATL','Median','Mean'])
        
        for value in close_values:

            aggregated_list.append(value)

            no_of_items = len(aggregated_list)

            if value > ATH:
                ATH = value
            elif value < ATL:
                ATL = value

            median_val = statistics.median(aggregated_list)
            avg_val = statistics.mean(aggregated_list)
            
            #print('At index number: {}'.format(no_of_items))

            #print("ATH: {}, ATL: {}, Running Median: {}, Running Mean: {}\n".format(ATH,ATL,median_val,avg_val))

            #date_val = date_values[no_of_items+1]
            self.aggregate_df.loc[no_of_items] = [value, ATH ,ATL, median_val, avg_val]
            
        '''
        
        '''    
        self.aggregate_df['SMA7'] = self.aggregate_df['Close'].rolling(window=7).mean()
        self.aggregate_df['SMA30'] = self.aggregate_df['Close'].rolling(window=30).mean()
        self.aggregate_df['SMA-MACD'] = self.aggregate_df['SMA7'] - self.aggregate_df['SMA30']
        self.aggregate_df['SMA-MACD-ratio'] = self.aggregate_df['SMA7'] / self.aggregate_df['SMA30']
        self.aggregate_df['bool_diff_SMA7_SMA30'] = self.aggregate_df['SMA-MACD'].apply(is_ma_pos)
        reversed_df = self.raw_df[['date']]
        date_list = reversed_df['date'][::-1].tolist()
        se = pd.Series(date_list)
        self.aggregate_df['date'] = se.values
        self.aggregate_df['name'] = self.token_name
        self.aggregate_df['close_diff'] = self.aggregate_df['Close'].diff()
        self.aggregate_df['bool_close_diff'] = self.aggregate_df['close_diff'].apply(is_ma_pos)
        display(self.aggregate_df)
        
        
    def test_sma_values(self):
        signal_list = [*range(1, 20, 1)] 
        ma_list = [*range(21, 50, 1)] 
        
        ATH = algo_close_values[0]
        ATL = algo_close_values[0]
        MEDIAN = 0
        AVERAGE = 0
        
        aggregated_list = []
        self.aggregate_df = pd.DataFrame(columns=['Close','ATH','ATL','Median','Mean'])
        
        for value in close_values:

            aggregated_list.append(value)

            no_of_items = len(aggregated_list)

            if value > ATH:
                ATH = value
            elif value < ATL:
                ATL = value

            median_val = statistics.median(aggregated_list)
            avg_val = statistics.mean(aggregated_list)
            
            #print('At index number: {}'.format(no_of_items))

            #print("ATH: {}, ATL: {}, Running Median: {}, Running Mean: {}\n".format(ATH,ATL,median_val,avg_val))

            #date_val = date_values[no_of_items+1]
            self.aggregate_df.loc[no_of_items] = [value, ATH ,ATL, median_val, avg_val]
            
        self.aggregate_df['SMA7'] = self.aggregate_df['Close'].rolling(window=7).mean()
        self.aggregate_df['SMA30'] = self.aggregate_df['Close'].rolling(window=30).mean()
        self.aggregate_df['SMA-MACD'] = self.aggregate_df['SMA7'] - self.aggregate_df['SMA30']
        self.aggregate_df['SMA-MACD-ratio'] = self.aggregate_df['SMA7'] / self.aggregate_df['SMA30']
        self.aggregate_df['bool_diff_SMA7_SMA30'] = self.aggregate_df['SMA-MACD'].apply(is_ma_pos)
        reversed_df = self.raw_df[['date']]
        date_list = reversed_df['date'][::-1].tolist()
        se = pd.Series(date_list)
        self.aggregate_df['date'] = se.values
        self.aggregate_df['name'] = self.token_name
        self.aggregate_df['close_diff'] = self.aggregate_df['Close'].diff()
        self.aggregate_df['bool_close_diff'] = self.aggregate_df['close_diff'].apply(is_ma_pos)        
        
        
    def visualize_sma(self):
        print("{} SMA7 and SM30 Chart".format(self.token_name))
        #display(self.aggregate_df.head(5))
        fig = px.line(self.aggregate_df, x = 'date', y=['Close','SMA7','SMA30'])
        fig.show()
        
    def calc_ema(self, window):
        
        col_name = 'EMA'+str(window)
        data = self.aggregate_df['Close'].to_numpy()
        
        alpha = 2 /(window + 1.0)
        alpha_rev = 1-alpha

        scale = 1/alpha_rev
        n = data.shape[0]

        r = np.arange(n)
        scale_arr = scale**r
        offset = data[0]*alpha_rev**(r+1)
        pw0 = alpha*alpha_rev**(n-1)

        mult = data*pw0*scale_arr
        cumsums = mult.cumsum()
        out = offset + cumsums*scale_arr[::-1]
        
        se = pd.Series(out)
        self.aggregate_df[col_name] = se.values
    
    def calc_ema_macd(self):
        self.aggregate_df['EMA-MACD'] = self.aggregate_df['EMA12'] - self.aggregate_df['EMA26']
        self.aggregate_df['EMA-MACD-ratio'] = self.aggregate_df['EMA12'] / self.aggregate_df['EMA26']
        self.aggregate_df['bool_diff_EMA12_EMA26'] = self.aggregate_df['EMA-MACD'].apply(is_ma_pos)
        
    def visualize_ema(self, EMA_col_name1,EMA_col_name2):
        print("{} {} and {} Chart".format(self.token_name, EMA_col_name1,EMA_col_name2))
        #display(self.aggregate_df.head(5))
        fig = px.line(self.aggregate_df, x = 'date', y=['Close',EMA_col_name1,EMA_col_name2])
        fig.show()
        
    def calc_confusion_matrix(self, sma_or_ema):
        
        if sma_or_ema == 'sma':
            # confusion matrix
            matrix = confusion_matrix(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_SMA7_SMA30'], labels=[1,0])
            print('Confusion matrix : \n',matrix)

            # outcome values order in sklearn
            tp, fn, fp, tn = confusion_matrix(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_SMA7_SMA30'],labels=[1,0]).reshape(-1)
            print('Outcome values : \n', tp, fn, fp, tn)

            # classification report for precision, recall f1-score and accuracy
            matrix = classification_report(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_SMA7_SMA30'],labels=[1,0])
            print('Classification report : \n',matrix)
            print(confusion_matrix)

            self.sma_model_accuracy = (tp + tn) / (tp+fn+fp+tn)
            print('Model accuracy score: {}'.format(self.sma_model_accuracy))
            
            return self.sma_model_accuracy
        
        else:
            # confusion matrix
            matrix = confusion_matrix(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_EMA12_EMA26'], labels=[1,0])
            print('Confusion matrix : \n',matrix)

            # outcome values order in sklearn
            tp, fn, fp, tn = confusion_matrix(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_EMA12_EMA26'],labels=[1,0]).reshape(-1)
            print('Outcome values : \n', tp, fn, fp, tn)

            # classification report for precision, recall f1-score and accuracy
            matrix = classification_report(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_EMA12_EMA26'],labels=[1,0])
            print('Classification report : \n',matrix)
            print(confusion_matrix)

            self.ema_model_accuracy = (tp + tn) / (tp+fn+fp+tn)
            print('Model accuracy score: {}'.format(self.ema_model_accuracy))
            
            return self.ema_model_accuracy

In [ ]:
algo_obj = token('ALGO-USD', 300)
algo_obj.calc_sma()
algo_obj.visualize_sma()

In [ ]:
algo_obj.calc_confusion_matrix('sma')

In [ ]:
algo_obj.calc_ema(12)
algo_obj.calc_ema(26)
algo_obj.calc_ema_macd()

In [ ]:
algo_obj.calc_confusion_matrix('ema')

In [ ]:
algo_obj.visualize_ema('EMA12', 'EMA26')

## Consolidated Token Analysis

In [ ]:
algo_obj = token('ALGO-USD', 300)
grt_obj = token('GRT-USD', 300)
btc_obj = token('BTC-USD',300)
eth_obj = token('ETH-USD',300)
aave_obj = token('AAVE-USD',300)
atom_obj = token('ATOM-USD',300)
bal_obj = token('BAL-USD',300)
band_obj = token('BAND-USD',300)
eos_obj = token('EOS-USD',300)
link_obj = token('LINK-USD',300)
ltc_obj = token('LTC-USD',300)
nu_obj = token('NU-USD',300)
omg_obj = token('OMG-USD',300)
uni_obj = token('UNI-USD',300)
xlm_obj = token('XLM-USD',300)
zrx_obj = token('ZRX-USD',300)

In [ ]:
token_list = [algo_obj, grt_obj, btc_obj, eth_obj, aave_obj, atom_obj, bal_obj, band_obj, eos_obj, link_obj, ltc_obj, nu_obj, omg_obj, uni_obj, xlm_obj, zrx_obj]

### Calculate and visualize simple moving averages

In [ ]:
for token in token_list:
    token.calc_sma()
    token.visualize_sma()

### Calculate and visualize exponential moving averages

In [ ]:
for token in token_list:
    token.calc_ema(12)
    token.calc_ema(26)
    token.calc_ema_macd()
    token.visualize_ema('EMA12','EMA26')

### Calculate confusion matrices and model accuracy scores

In [ ]:
ema_score_list = []
token_name_list = []
for token in token_list:
    print("{} Confusion Matrix".format(token.token_name))
    token_ema = token.calc_confusion_matrix('ema')
    ema_score_list.append(token_ema)
    
    t_name = token.token_name
    token_name_list.append(t_name)

In [ ]:
complete_token_ema_accuracy = list(zip(token_name_list,ema_score_list))
print(tuple(complete_token_ema_accuracy))

In [ ]:
df = pd.DataFrame(complete_token_ema_accuracy, columns=['Token-Name','Model Decision Accuracy'])
df['Model Decision Accuracy'] = df['Model Decision Accuracy'] *100 
df

In [ ]:
grt_df = grt_obj.return_df()
grt_df

In [ ]:
signal_list = [2,3,4,5,6,7,8,9,10] 
ma_list = [20,21,22,23,24,25,26,27,28,29,30] 



In [ ]:

for signal_val in signal_list:
    for ma_val in ma_list:
        algo_obj.calc_ema(signal_val)
        algo_obj.calc_ema(ma_val)
        algo_obj.calc_ema_macd()
        token_ema = algo_obj.calc_confusion_matrix('ema')

In [ ]:
def calc_ema_array(data,window):
    alpha = 2 /(window + 1.0)
    alpha_rev = 1-alpha

    scale = 1/alpha_rev
    n = data.shape[0]

    r = np.arange(n)
    scale_arr = scale**r
    offset = data[0]*alpha_rev**(r+1)
    pw0 = alpha*alpha_rev**(n-1)

    mult = data*pw0*scale_arr
    cumsums = mult.cumsum()
    out = offset + cumsums*scale_arr[::-1]

    return out

In [ ]:
class token:
    
    def __init__(self, name, num_of_days):
        
        print("Creating {} object ...".format(name))
        self.REST_API = 'https://api.pro.coinbase.com'
        self.PRODUCTS = REST_API+'/products'
        
        self.token_name = name
        self.token_url = '/'+name+'/candles'
        
        self.start_date = (datetime.today() - timedelta(days=300)).isoformat()
        self.end_date = datetime.now().isoformat()
        
        # Please refer to the coinbase documentation on the expected parameters
        self.params = {'start':self.start_date, 'end':self.end_date, 'granularity':'86400'}
        self.response = connect(PRODUCTS+ self.token_url, param = params)
        
        self.response_text = self.response.text
        
        self.raw_df = pd.read_json(self.response_text)
        
        self.raw_df.columns = ['time','low','high','open','close','volume']
        self.raw_df['date'] = pd.to_datetime(self.raw_df['time'], unit='s')
        self.raw_df['year'] = pd.DatetimeIndex(self.raw_df['date']).year
        self.raw_df['month'] = pd.DatetimeIndex(self.raw_df['date']).month
        self.raw_df['day'] = pd.DatetimeIndex(self.raw_df['date']).day
        self.raw_df['hour'] = pd.DatetimeIndex(self.raw_df['date']).hour.astype(str)
        self.raw_df['name'] = self.token_name
        #self.raw_df.set_index('date', inplace=True)
        
        self.sma_signal = 7
        self.sma_ma = 25
        self.ema_signal = 12
        self.ema_ma = 26
        self.sma_model_accuracy = 0
        self.ema_model_accuracy = 0
        
    def return_df(self):
        
        return self.aggregate_df
    

    def calc_sma(self):
        
        close_values = self.raw_df['close'][::-1].tolist()
    
        
        INVESTMENT = 100 # initial investment
        NO_OF_RECORDS = 0 # number of transactions
        ATH = algo_close_values[0]
        ATL = algo_close_values[0]
        MEDIAN = 0
        AVERAGE = 0
        
        aggregated_list = []
        self.aggregate_df = pd.DataFrame(columns=['Close','ATH','ATL','Median','Mean'])
        
        for value in close_values:

            aggregated_list.append(value)

            no_of_items = len(aggregated_list)

            if value > ATH:
                ATH = value
            elif value < ATL:
                ATL = value

            median_val = statistics.median(aggregated_list)
            avg_val = statistics.mean(aggregated_list)
            
            #print('At index number: {}'.format(no_of_items))

            #print("ATH: {}, ATL: {}, Running Median: {}, Running Mean: {}\n".format(ATH,ATL,median_val,avg_val))

            #date_val = date_values[no_of_items+1]
            self.aggregate_df.loc[no_of_items] = [value, ATH ,ATL, median_val, avg_val]
            
        '''
        
        '''    
        self.aggregate_df['SMA7'] = self.aggregate_df['Close'].rolling(window=7).mean()
        self.aggregate_df['SMA30'] = self.aggregate_df['Close'].rolling(window=30).mean()
        self.aggregate_df['SMA-MACD'] = self.aggregate_df['SMA7'] - self.aggregate_df['SMA30']
        self.aggregate_df['SMA-MACD-ratio'] = self.aggregate_df['SMA7'] / self.aggregate_df['SMA30']
        self.aggregate_df['bool_diff_SMA7_SMA30'] = self.aggregate_df['SMA-MACD'].apply(is_ma_pos)
        reversed_df = self.raw_df[['date']]
        date_list = reversed_df['date'][::-1].tolist()
        se = pd.Series(date_list)
        self.aggregate_df['date'] = se.values
        self.aggregate_df['name'] = self.token_name
        self.aggregate_df['close_diff'] = self.aggregate_df['Close'].diff()
        self.aggregate_df['bool_close_diff'] = self.aggregate_df['close_diff'].apply(is_ma_pos)
        display(self.aggregate_df)
        
        
    def test_sma_values(self):
        signal_list = [*range(1, 20, 1)] 
        ma_list = [*range(21, 50, 1)] 
        
        ATH = algo_close_values[0]
        ATL = algo_close_values[0]
        MEDIAN = 0
        AVERAGE = 0
        
        aggregated_list = []
        self.aggregate_df = pd.DataFrame(columns=['Close','ATH','ATL','Median','Mean'])
        
        for value in close_values:

            aggregated_list.append(value)

            no_of_items = len(aggregated_list)

            if value > ATH:
                ATH = value
            elif value < ATL:
                ATL = value

            median_val = statistics.median(aggregated_list)
            avg_val = statistics.mean(aggregated_list)
            
            #print('At index number: {}'.format(no_of_items))

            #print("ATH: {}, ATL: {}, Running Median: {}, Running Mean: {}\n".format(ATH,ATL,median_val,avg_val))

            #date_val = date_values[no_of_items+1]
            self.aggregate_df.loc[no_of_items] = [value, ATH ,ATL, median_val, avg_val]
            
        self.aggregate_df['SMA7'] = self.aggregate_df['Close'].rolling(window=7).mean()
        self.aggregate_df['SMA30'] = self.aggregate_df['Close'].rolling(window=30).mean()
        self.aggregate_df['SMA-MACD'] = self.aggregate_df['SMA7'] - self.aggregate_df['SMA30']
        self.aggregate_df['SMA-MACD-ratio'] = self.aggregate_df['SMA7'] / self.aggregate_df['SMA30']
        self.aggregate_df['bool_diff_SMA7_SMA30'] = self.aggregate_df['SMA-MACD'].apply(is_ma_pos)
        reversed_df = self.raw_df[['date']]
        date_list = reversed_df['date'][::-1].tolist()
        se = pd.Series(date_list)
        self.aggregate_df['date'] = se.values
        self.aggregate_df['name'] = self.token_name
        self.aggregate_df['close_diff'] = self.aggregate_df['Close'].diff()
        self.aggregate_df['bool_close_diff'] = self.aggregate_df['close_diff'].apply(is_ma_pos)        
        
        
    def visualize_sma(self):
        print("{} SMA7 and SM30 Chart".format(self.token_name))
        #display(self.aggregate_df.head(5))
        fig = px.line(self.aggregate_df, x = 'date', y=['Close','SMA7','SMA30'])
        fig.show()
        

        
    def calc_ema(self, window1, window2):
        
        self.col1_name = 'EMA'+str(window1)
        self.col2_name = 'EMA'+str(window2)
        data = self.aggregate_df['Close'].to_numpy()
        
        alpha1 = 2 /(window1 + 1.0)
        alpha_rev1 = 1-alpha1

        scale1 = 1/alpha_rev1
        n1 = data.shape[0]

        r1 = np.arange(n1)
        scale_arr1 = scale1**r1
        offset1 = data[0]*alpha_rev1**(r1+1)
        pw01 = alpha1*alpha_rev1**(n1-1)

        mult1 = data*pw01*scale_arr1
        cumsums1 = mult1.cumsum()
        out1 = offset1 + cumsums1*scale_arr1[::-1]

        #
        alpha2 = 2 /(window2 + 1.0)
        alpha_rev2 = 1-alpha2

        scale2 = 1/alpha_rev2
        n2 = data.shape[0]

        r2 = np.arange(n2)
        scale_arr2 = scale2**r2
        offset2 = data[0]*alpha_rev2**(r2+1)
        pw02 = alpha2*alpha_rev2**(n2-1)

        mult2 = data*pw02*scale_arr2
        cumsums2 = mult2.cumsum()
        out2 = offset2 + cumsums2*scale_arr2[::-1]


        
        se1 = pd.Series(out1)
        se2 = pd.Series(out2)
        self.aggregate_df[self.col1_name] = se1.values
        self.aggregate_df[self.col2_name] = se2.values
        
    def calc_ema_macd(self):
        self.aggregate_df['EMA-MACD'] = self.aggregate_df[self.col1_name] - self.aggregate_df[self.col2_name]
        self.aggregate_df['EMA-MACD-ratio'] = self.aggregate_df[self.col1_name] / self.aggregate_df[self.col2_name]
        self.aggregate_df['bool_diff_signal_ma'] = self.aggregate_df['EMA-MACD'].apply(is_ma_pos)
        
    def visualize_ema(self, EMA_col_name1,EMA_col_name2):
        print("{} {} and {} Chart".format(self.token_name, EMA_col_name1,EMA_col_name2))
        #display(self.aggregate_df.head(5))
        fig = px.line(self.aggregate_df, x = 'date', y=['Close',EMA_col_name1,EMA_col_name2])
        fig.show()
        
    def calc_confusion_matrix(self, sma_or_ema):
        
        if sma_or_ema == 'sma':
            # confusion matrix
            matrix = confusion_matrix(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_SMA7_SMA30'], labels=[1,0])
            print('Confusion matrix : \n',matrix)

            # outcome values order in sklearn
            tp, fn, fp, tn = confusion_matrix(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_SMA7_SMA30'],labels=[1,0]).reshape(-1)
            print('Outcome values : \n', tp, fn, fp, tn)

            # classification report for precision, recall f1-score and accuracy
            matrix = classification_report(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_SMA7_SMA30'],labels=[1,0])
            print('Classification report : \n',matrix)
            print(confusion_matrix)

            self.sma_model_accuracy = (tp + tn) / (tp+fn+fp+tn)
            print('Model accuracy score: {}'.format(self.sma_model_accuracy))
            
            return self.sma_model_accuracy
        
        else:
            # confusion matrix
            matrix = confusion_matrix(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_signal_ma'], labels=[1,0])
            print('Confusion matrix : \n',matrix)

            # outcome values order in sklearn
            tp, fn, fp, tn = confusion_matrix(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_signal_ma'],labels=[1,0]).reshape(-1)
            print('Outcome values : \n', tp, fn, fp, tn)

            # classification report for precision, recall f1-score and accuracy
            matrix = classification_report(self.aggregate_df['bool_close_diff'],self.aggregate_df['bool_diff_signal_ma'],labels=[1,0])
            print('Classification report : \n',matrix)
            print(confusion_matrix)

            self.ema_model_accuracy = (tp + tn) / (tp+fn+fp+tn)
            print('Model accuracy score: {}'.format(self.ema_model_accuracy))
            
            return self.ema_model_accuracy

In [ ]:
signal_list = [2,3,4,5,6,7,8,9,10] 
ma_list = [20,21,22,23,24,25,26,27,28,29,30] 



In [ ]:
algo_obj = token('ALGO-USD', 300)
for signal_val in signal_list:
    for ma_val in ma_list:
        algo_obj.calc_sma()
        algo_obj.calc_ema(signal_val, ma_val)
        algo_obj.calc_ema_macd()
        algo_obj.calc_confusion_matrix('ema')